In [1]:
from modules.llm import *
from modules.utils import *

In [2]:
config = {
    "rqa_prompt_template" : "This database is a list of dataset metadata. Use the following pieces of context to find the relevant document. Answer only from the context given using the {question} given. If you do not know the answer, say you do not know. {context}",
    "num_return_documents" : 50,
    "embedding_model": "BAAI/bge-base-en-v1.5",
    "llm_model": "HuggingFaceH4/zephyr-7b-beta",
    "persist_dir": "./chroma_db/",
    "recreate_chroma": False,
}

In [3]:
metadata_df, all_dataset_metadata = create_metadata_dataframe(*get_all_dataset_metadata_from_openml())
metadata_df = clean_metadata_dataframe(metadata_df)
print(metadata_df.head())


   did                                        description
0    2  **Author**: Unknown. Donated by David Sterling...
1    3  Author: Alen Shapiro\nSource: [UCI](https://ar...
2    4  **Author**: Unknown\n**Source**: Collective Ba...
3    5  **Author**: H. Altay Guvenir, Burak Acar, Hald...
4    6  **Author**: David J. Slate  \n**Source**: [UCI...


In [4]:
vectordb = load_document_and_create_vector_store(metadata_df, model_name=config['embedding_model'], recreate_chroma=config['recreate_chroma'], persist_directory=config['persist_dir'])

/Users/eragon/.pyenv/versions/3.9.19/envs/openml/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
retriever, llm = create_retriever_and_llm(vectordb,num_return_documents=config["num_return_documents"], model_repo_id=config["llm_model"])
qa = create_llm_chain_and_query(vectordb=vectordb,retriever=retriever,llm=llm, prompt_template = config["rqa_prompt_template"])

/Users/eragon/.pyenv/versions/3.9.19/envs/openml/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [20]:
# %time
# query = "Which datasets would be useful for stock market support?"
# query = "Which datasets would be useful for heart disease"
# query = "Which datasets would be useful for flowers"
# query = "Which datasets would be useful for image classification"
# query = "My supervisor wants me to work on cloud cover, which datasets can I use"
# query = "Are there any datasets from the netherlands?"
query = "Are there any datasets about farm animals?"
# query = "Find chinese authors"
results = create_result_dataframe(query, qa, all_dataset_metadata)

In [21]:
results['description'].values[:10]

array(['**Author**: Richard S. Forsyth   \n**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/Zoo) - 5/15/1990   \n**Please cite**:  \n\n**Zoo database**  \nA simple database containing 17 Boolean-valued attributes describing animals.  The "type" attribute appears to be the class attribute. \n\nNotes:  \n* I find it unusual that there are 2 instances of "frog" and one of "girl"!\n* feature \'animal\' is an identifier (though not unique) and should be ignored when modeling',
       'Data Set Information:',
       "**Author**: B.D. Ripley  \n**Source**: StatLib - Date unknown  \n**Please cite**:   \n\nDataset from `Pattern Recognition and Neural Networks' by B.D. Ripley. Cambridge University Press (1996)  ISBN  0-521-46086-7\n\nThe background to the datasets is described in section 1.4; this file relates the computer-readable files to that description.\n\nviruses\n\nThis is a dataset on 61 viruses with rod-shaped particles affecting various crops (tobacco, tomato, cucumber and oth

In [11]:
results['name'].values[:10]

array(['covertype', 'sylva_agnostic', 'test_dataset', 'Domainome',
       'dataset_sales', 'dataset-autoHorse_fixed', 'Dataset',
       'RelevantImagesDatasetTEST', 'MY_DB', 'Laboratorio_dataset_car'],
      dtype=object)